# **Demo: Text Summarizer**

This demo guides you through the process of creating an Arxiv paper summarizer that automates downloading a research paper as a PDF, extracting its content, and generating a concise summary. For this demonstration, the example paper titled "The Impact of Generative Artificial Intelligence" will be used. The summarizer processes the document, extracts key insights, and delivers a structured summary using LangChain.

##**Steps to Perform:**

*   Step 1: Import the Necessary Libraries
*   Step 2: Download and Read the PDF
*   Step 3: Extract Text from the PDF
*   Step 4: Count the Tokens in the Extracted Text
*   Step 5: Use LangChain to Generate a Summary of the Paper



###**Step 1: Import the Necessary Libraries**

In [ ]:
import os
import requests
from PyPDF2 import PdfReader
import tiktoken
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback

Number of pages: 9


###**Step 2: Download and Read the PDF**

*   Define the path of the paper.
*   Read the PDF using **PdfReader**.
*   Print the number of pages.



In [ ]:
# Define the path of the paper
PAPER_PATH = "arxiv_impact_of_GENAI.pdf"

# Read the PDF
reader = PdfReader(PAPER_PATH)

# Print the number of pages in the PDF
print(f"Number of pages: {len(reader.pages)}")

###**Step 3: Extract Text from the PDF**

*   Initialize an empty list to store text parts.
*   Define a function to visit the body of the text.
*   Extract the text from each page of the PDF.
*   Join the parts of the text into a single string.
*   Print the extracted text.




In [ ]:
# Initialize an empty list to store the parts of the text
parts = []

# Define a function to visit the body of the text
def visitor_body(text, cm, tm, fontDict, fontSize):
    y = tm[5]
    if y > 50 and y < 720:
        parts.append(text)

# Extract the text from each page of the PDF
for page in reader.pages:
    page.extract_text(visitor_text=visitor_body)

# Join the parts of the text into a single string
text_body = "".join(parts)

# Print the extracted text
print(text_body)


###**Step 4: Count the Tokens in the Extracted Text**

*   Define a function to count the tokens in a text string.
*   Count the tokens in the extracted text.
*   Print the number of tokens.



In [ ]:
# Define a function to count the tokens in a text string
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Count the tokens in the extracted text
num_tokens = num_tokens_from_string(text_body, "gpt-3.5-turbo")

# Print the number of tokens
print(num_tokens)

###**Step 5: Use LangChain to Generate a Summary of the Paper**

*   Define the system and human prompts.
*   Create the **ChatPromptTemplate** object.
*   Create the **ChatOpenAI** object.
*   Create the **LLMChain** object.
*   Run the **LLMChain**.
*   Print the output.




In [ ]:
# Define the system prompt
context_template = "You are a helpful AI Researcher that specializes in analyzing ML, AI, and LLM papers. Please use all your expertise to approach this task. Output your content in markdown format and include titles where relevant."
system_message_prompt = SystemMessagePromptTemplate.from_template(context_template)

# Define the human prompt
human_template = "Please summarize this paper focusing on the key important takeaways for each section. Expand the summary on methods so they can be clearly understood. \n\n PAPER: \n\n{paper_content}"
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=human_template,
            input_variables=["paper_content"],
        )
    )

# Create the ChatPromptTemplate object
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Create the ChatOpenAI object
chat = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0.2)

# Create the LLMChain object
summary_chain = LLMChain(llm=chat, prompt=chat_prompt_template)

# Run the LLMChain and print the output
with get_openai_callback() as cb:
    output = summary_chain.run(text_body)
print(output)

###**Conclusion**

This demo provides a step-by-step guide on how to build an Arxiv paper summarizer using Python. By following these steps.

**Note:**
*   Save the output as a text file.
*   Name the text file **Summary.txt**.
*   This file will be used in the next session for benchmarking purposes.

